# Planning small TWEAC Setup for physics and system tests

static constexpr float_64 TDELAY = 0.; //66.7e-6 / SI::SPEED_OF_LIGHT_SI;Setup of a 3D simulation of a Travelling-Wave Electron Acceleration setup, where two laser pulses "collide" within an underdense plasma.

Both laser pulses are linearly polarized in $x$, propagate in the $yz$-plane, and feature 800\,nm central wavelength, 10\,fs (FWHM) transform-limited pulse duration, and a peak intensity of $2.62 \times 10^{19}$\,W/cm$^2$ ($a_0 = 3.5$), respectively.
Their propagation direction encloses an angle of $3.5^\circ$ and $-3.5^\circ$ with respect to the y axis.
The focal geometry is cylindrical, and the focal lines both coincide with the y axis.
The laser pulse focal line height is $w_\mathrm{0;x} = 1.2$\,μm.
Since the focal line length $w_\mathrm{0;yz}$ easily extends beyond the centimeter range, we assume, for the sake of simplicity, that the laser is unbounded in the yz plane of the simulation.
The pulse-front tilts are $\alpha = \pm 1.75^\circ$, respectively.
The angle enclosed by the propagation directions of the lasers is $7^\circ$.

The original publication from Alex:

Alexander Debus, Richard Pausch, Axel Huebl, Klaus Steiniger, René Widera, Thomas E. Cowan, Ulrich Schramm, and Michael Bussmann.
**Circumventing the Dephasing and Depletion Limits of Laser-Wakefield Acceleration**.
*Phys. Rev. X 9, 031044*.
(https://doi.org/10.1103/PhysRevX.9.031044)

## Gas params

* electron density linearly rises from vacuum to
  $6.4 \times 10^{18}\,\mathrm{cm}^{-3}$ within a short distance of 20\,μm.
  After this initial shock, it linearly decreases on a longer scale,
  first to $4.5 \times 10^{18}\,\mathrm{cm}^{-3}$ and finally to
  $3.2 \times 10^{18}\,\mathrm{cm}^{-3}$ over
  subsequent distances of 160\,μm and 460\,μm

* free electron density in initial shock region (=BASE_DENSITY) in critical densities = 3.67e-03n_c

* no initial temperature

* no ions


## External electron params


## Laser params

* Linearly $x$-polarized fastTWTS of width $w_\mathrm{0;x} = 1.2$\,μm

* propagation axis in $yz$-plane and enclosing an angle of $\pm 3.5^\circ $ with y axis

* wavelength λ_Laser = 800.0nm

* a_0 = 3.5

* FWHM duration of laser temporal intensity envelope = 10.000fs (=> standard deviation σ_I = 4.247fs)

* Field initialization starts at the simulation domain wall 4.5σ_I before field maximum.

* Critical density nc = 1.742e+27/m^3 = 1.742e+21/cm^3

* **What is the effect of the focus_y variable, wo doch the laser extdends infintely in z direction such that there is no "laser center"**


## Grid params

* Δx = Δy = Δz = λ_Laser / 8.

* Δt = 0.1 * 0.99 * Δx / c / sqrt(3.) **Reevaluate this choice ones the stability condition is known.**

* **This is a very low resolution, the choice is motivated by the use of the higher order solver.
  How does the cold plasma instability affect our simulation setup?
  The minimum electron temperature to avoid the instability is 1.3keV...not a lot, isn't it?
  Do we need to increase resolution to mitigate the instability?**

## Particle & Field solver params

* Particle shape: PQS (s. Hockney p. 144)

* Particle pusher: Higuera-Cary

* No ionization methods (for the moment)

* No collisions

* homogenous macro-particle density of 25 macro-particles/cell

* minimum macro-particle weighting: 10

* particles are initially randomly distributed within a cell

* Current solver: EmZ

* `using CurrentInterpolation = currentInterpolation::None;`

* Field solver: `ArbitraryOrderFDTDPML< 4, CurrentInterpolation >;` **See if we can change to 8 neighbors on newer machines**

* Perfectly matched layer absorbing boundary conditions at every simulation domain wall

## Simulation params

* 3D simulation

* simulation domain length along x = 608 cells on 2 GPUs

* simulation domain length along y (moving window propagation direction) = 864 cells on 4 GPUs
  (remember there is one row of GPUs missing due to moving window)

* simulation domain length along z = 1056 cells on 2 GPUs

* simulation time = 650000Δt **?**

* **TODO: Set window movePoint appropriate!**


## Imports

In [ ]:
%pylab inline

import scipy.constants as sc

import sys

sys.path.append("../lib/python/")
from picongpu.utils import MemoryCalculator

In [ ]:
# OVERWRITE DEFAULT PLOTTING PARAMETERS
params = {
    'font.family' : 'sans-serif',
    'font.sans-serif' : ['DejaVu Sans', 'Arial', 'Verdana', 'Liberation Sans'],
    'mathtext.default' : 'regular',
    'mathtext.rm' : 'sans',
    'font.size' : 18,
    'legend.fontsize' : 14,
    'axes.labelsize' : 18,
    'axes.titlesize' : 16,
    'lines.linewidth' : 3.0,
    'legend.frameon' : False,
    'legend.numpoints': 1,
}

matplotlib.rcParams.update(params)

# Function definitions

In [ ]:
from scipy.optimize import newton


def critical_density_si(lambda_laser):
    """ Critical electron number density at which an 
        incident light wave is reflected from a surface.
        Units [1/m^3]
    """
    return (sc.epsilon_0*sc.m_e/sc.e**2)*(2.*pi*sc.c/lambda_laser)**2


def critical_density(lambda_laser):
    """ Critical electron number density in [1/cm^3]
    """
    return critical_density_si(lambda_laser)*1.e-6


def plasma_angular_frequency(n_e):
    """ 
    """
    return sqrt( n_e * sc.e**2 / ( sc.epsilon_0 * sc.m_e ) )


def plasma_oscillation_length(n_e):
    """ Assuming the driver propagates at the speed of light
    """
    omega_p = plasma_angular_frequency(n_e)
    return 2.*sc.pi*sc.c/omega_p


def debye_length(n_e, T , Z):
    """ electron temperature [T] = eV
        electron density [n_e] = cm^-3
    """
    return 743.*sqrt(T/(1.+Z)/n_e)  # sqrt(epsilon_0*k_B*T/n_e/e_charge**2/(1.+Z))


def ghrist_b0(n_h, k):
        """ Coefficients in the stencil of the arbitrary-order finite-difference on a staggered grid.
            Taken from Michelle Ghrist's PhD thesis.
            Return coefficient for k'th neighbor at spatial derivative order 2*n_h.
        """
        n_i = int(abs(n_h))
        k_i = int(2.*abs(k))/2.
        assert k_i != 0, "k_i != 0 required"
        b0_n_k = 4.*n_h*( np.math.factorial(2*n_h) / ( 2**(2*n_h) * ( np.math.factorial(n_h) )**2 )  )**2
        for k_l in np.arange(3/2, k_i+1, 1):
            b0_n_k *= (-1.) * ( k_l - 1. )**2 / k_l**2 * ( 2*n_i + 1. - 2.*k_l ) / ( 2*n_i - 1. + 2.*k_l )
        return b0_n_k if ( k > 0 ) else ( -b0_n_k )

    
def dispersion_rel_2D(k, ϕ, N_λ, S, n_h):
    k_part = np.zeros_like(k)
    for l in np.arange(0.5, n_h-0.5 + 1, 1):
        for n in np.arange(0.5, n_h-0.5 + 1, 1):
            k_part += ghrist_b0(n_h, l) * ghrist_b0(n_h, n) * ( 
                np.sin( l * k * np.cos(ϕ) ) * np.sin( n * k * np.cos(ϕ) )
                +  np.sin( l * k * np.sin(ϕ) ) * np.sin( n * k * np.sin(ϕ) )
            )
    k_part -= ( np.sin( sc.pi * S / N_λ ) / S )**2
    return k_part


def dispersion_wave_num_ϕ(ϕ, N_λ, S, n_h):
    return np.array([ newton(dispersion_rel_2D, x0=1., args=(ϕ_i, N_λ, S, n_h) ) for ϕ_i in ϕ ])


def dispersion_wave_num_S(ϕ, N_λ, S, n_h):
    return np.array([ newton(dispersion_rel_2D, x0=1., args=(ϕ, N_λ, S_i, n_h) ) for S_i in S ])


def dispersion_wave_num_N_λ(ϕ, N_λ, S, n_h):
    return np.array([ newton(dispersion_rel_2D, x0=1., args=(ϕ, N_λ_i, S, n_h) ) for N_λ_i in N_λ ])


# General considerations on material and resolution

## Laser parameters

In [ ]:
# LASER

## Change w_0x to Rayleighlength >~ Plasma oscillation length
## Change pulse duration to tau_FWHM,I = 30fs -> sigma_t,E = tau_FWHM,I / sqrt(2. * log(2.)) / sqrt(2)
## sigma_t,E = tau_FWHM,I / 1.6651 = 18.0168
## lower a0 to just enter the bubble regime

λ_0 = 800e-9 # laser wavelength
print("Laser period = %.1es"%(λ_0/sc.c))

_A0 = 3.5
print("Laser a_0 = %.2f"%(_A0))

τ_FWHM_I = 10.e-15
print("FWHM duration of laser intensity = %.3ffs"%(τ_FWHM_I * 1.e15))
σ_I = τ_FWHM_I / ( 2.*sqrt( 2. * log(2.) ) )
print("σ_I of laser intensity = %.3ffs"%(σ_I * 1.e15))

ϕ = 5. * sc.pi / 180.
print("ϕ = %.1fdeg"%(ϕ*180./pi))

# beam waist in focus
w_0x = 5.e-6
print("w_0 = {:.2f} µm".format(w_0x * 1e6))

# Rayleigh length
z_r = 1 * np.pi * w_0x**2 / λ_0
print("z_r = {:.2f} µm".format(z_r * 1e6))

In [ ]:
n_c = critical_density_si(λ_0) # unit: ELEMENTS/m^3
print("critical e density for λ_Laser = %.3e/m^3"%(n_c))
print("plasma frequency at critical density = %.1erad/s"%(plasma_angular_frequency(n_c)))
print("plasma period at critical density = %.1es (= laser period)"%(2.*sc.pi/plasma_angular_frequency(n_c)))

## Plasma parameters

In [ ]:
BASE_DENSITY_SI = 2.5e+24 # unit: ELEMENTS/m^3
print("BASE_DENSITY = %.2en_c"%(BASE_DENSITY_SI / n_c))
PLATEAU_DENSITY_SI = BASE_DENSITY_SI
print("PLATEAU_DENSITY = %.2en_c"%(PLATEAU_DENSITY_SI / n_c))

In [ ]:
PLASMA_FREQUENCY_SI = plasma_angular_frequency(PLATEAU_DENSITY_SI)
print("plasma frequency at plateau density = %.1erad/s"%(PLASMA_FREQUENCY_SI))
print("plasma period at plateau density = %.1es"%(2.*sc.pi/PLASMA_FREQUENCY_SI))
λ_plasma = plasma_oscillation_length(PLATEAU_DENSITY_SI)
print("plasma oscillation length at plateau density = %.1fmum"%(λ_plasma*1.e6))
print("plasma skin depth = %.1fmum"%(sc.c/PLASMA_FREQUENCY_SI*1.e6))

In [ ]:
λ_plasma_base = plasma_oscillation_length(BASE_DENSITY_SI)
print("plasma frequency at base density = %.1erad/s"%(plasma_angular_frequency(BASE_DENSITY_SI)))
print("plasma oscillation length at base density = %.1fmum"%(λ_plasma_base*1.e6))

In [ ]:
print("Rayleighlength / plasma oscillation length =", z_r / λ_plasma)
print("Rayleighlength / base plasma oscillation length =", z_r / λ_plasma_base)

Looking at the plasma oscillation lengths, it becomes clear that the laser period is the smallest structure that should be resolved.

## Mesh parameters

As we use the 16th (or 8th) order FDTD solver, we are able to choose a low resolution of 8 cells/$\lambda_\mathrm{Laser}$ and keep the numerical dispersion error low with a time step of a tenth of the CFL limit. See plots in PIConGPU docu or field_solver notebook.

In [ ]:
dx_old = 1.60000e-7
dy_old = 2.00000e-8
dz_old = dx_old

dt_old = 6.536577e-18

n_e_old = 2.5e24 # unit: ELEMENTS/m^3
λ_plasma_old = plasma_oscillation_length(n_e_old)

print("λ_plasma = %.5fμm"%(λ_plasma_old*1.e6))

# 1120 2304 6144
Lx_old = dx_old*1120
Ly_old = dy_old*2304
Lz_old = dz_old*6144    # Because plasma should be "quiet" at the boundary and
                        # reflections where a problem and
                        # side cavities interesting

N_steps_old = 310000

print("λ_plasma / dx =", λ_plasma_old / dx_old )
print("λ_Laser / dy =", λ_0 / dy_old )
print("λ_plasma / dz =", λ_plasma_old / dz_old )
print("x Ausdehnung", Lx_old*1.e6, "μm =", Lx_old / λ_plasma_old, "λ_plasma =", Lx_old / w_0x, "w_0x")
print("y Ausdehnung", Ly_old*1.e6, "μm =", Ly_old / λ_plasma_old, "λ_plasma")
print("z Ausdehnung", Lz_old*1.e6, "μm =", Lz_old / λ_plasma_old, "λ_plasma =", Lz_old / z_r, "z_r")
z_off = Lz_old/2 * sin(ϕ)
print("Propagation distance of the laser along the focal mirror axis = %.2fµm = %.2fz_r"%(z_off*1.e6, z_off))
w_x_boundary = w_0x * sqrt(1. + (Lz_old/2)**2 / z_r**2)
print("Size at simulation domain boundary = %.2fµm, size of 99%% power aperture = %.2fLx"%(w_x_boundary*1.e6, w_x_boundary*sc.pi/Lx_old))

In [ ]:
# box length in y direction, from experience with Piz Daint simulation
L_y = 8. * λ_plasma_base
print("box length in y: {:.2f} µm".format(L_y * 1e6))

# box length in z direction, from experience with Piz Daint simulation (the laser pulse is not bounded in z-direction)
L_z = 15. * λ_plasma_base
print("box length in z: %.2f µm = %.2f λ_plasma = %.2f z_Rx"%(L_z * 1e6, L_z / λ_plasma, L_z/z_r))

# box length in x direction,
# from the typical criterion for 99% of beam power: pi * w for transversal size
# should fit into the simulation in order to properly model the propagation of the focusing field.
w_init = w_0x * np.sqrt(1 + (.5*L_z / z_r)**2)
print("w_init = {:.2f} µm".format(w_init * 1e6))
print(r"π * w_init = {:.2f} µm".format(np.pi * w_init * 1e6))

# take even more than maybe required from defocusing, also fit two plasma wavelength in the x-direction
L_x = 8* λ_plasma_base #40.e-6 # m
print("box length in x: {:.2f} µm".format(L_x * 1e6))

### Strict constraints

#### Transverse and longitudinal sampling

In [ ]:
## The cell length in the longitudinal direction is given by the laser wavelength
λ_0_proj_y = λ_0 / cos(ϕ)
dy_strict = λ_0_proj_y / 12.
print("dy = {:.5e} m".format(dy_strict))
print("sanity check: longitudinal samples per laser wavelength λ_laser: %.2f"%(λ_0/dy_strict))

In [ ]:
## If the projected laser period in the transverse direction is longer than the plasma oscillation length,
## I choose a cell size with equal length in x and z direction.
λ_0_proj_z = λ_0 / sin(ϕ)
print("Transverse laser oscillation length / plasma oscillation length =", λ_0_proj_z / λ_plasma)

In [ ]:
## Choose dz equal to dy, since the laser propagates more in z than in y direction
dz_strict = λ_0_proj_z / 12.
#dz_strict = λ_plasma_base / 8.
#print("sanity check: transverse samples per laser wavelength λ_laser: %.2f"%(λ_0/dz_strict))

In [ ]:
## The cell length in the transverse directions is given by Courant-Friedrich-Levy condition (eq. (4.98) in Taflove)
## and needs to be much smaller than the plasma oscillation length.
## A large aspect ratio is also not good since we will end up with a lot of particles per cell in the region
## of the accelerated bunch.
## Furthermore, in the sheeth region of the bubble we expect very high density, maybe 1-2 orders of magnitude higher
## than in the base density, and we do not want to model this region completely incorrect.
## Therefore we assume a maximum factor for the aspect ratio.
dx_strict = dz_strict #λ_plasma / 80.
print("dx = %.5em"%(dx_strict))
print("Samples per plasma oscillation length λ_p = 2πc/ω_p: %.2f"%(λ_plasma/dx_strict))

In [ ]:
## See how spatial and temporal discretization affects dispersion along the laser propagation direction
# CFL condition for AOFDTD with 4 neighbors
n_h = 4
aofdtd_coeffs = np.array([ghrist_b0(n_h, k+.5) for k in arange(n_h)])
print("AOFDTD coefficients = ", aofdtd_coeffs)
aofdtd = sum(np.array([(-1)**k * aofdtd_coeffs[k] for k in arange(n_h)]))
print("AOFDTD correction factor to Yee CFL = %f"%(aofdtd))

dt_max = 0.995 / aofdtd / sc.c / sqrt(1./dx_strict**2 + 1./dy_strict**2 + 1./dz_strict**2)


dt_range = dt_max * np.linspace(.05, 1., 20)

In [ ]:
def omega(wavenumber, angle, n_h, Δt, Δx, Δy, Δz):
    sqrt_x = sum(np.array([ghrist_b0(n_h, l) * sin(wavenumber*cos(angle)*l*Δx) / Δx for l in (arange(n_h)+.5)]))
    sqrt_y = sum(np.array([ghrist_b0(n_h, l) * sin(wavenumber*sin(angle)*l*Δy) / Δy for l in (arange(n_h)+.5)]))
    sqrt_z = 0.
    ξ = sc.c * Δt * sqrt(sqrt_x**2 + sqrt_y**2 + sqrt_z**2)
    assert all(abs(val) <= 1 for val in ξ), "Don't Do Bullshit!"
    return 2.*arcsin(ξ)/Δt


In [ ]:
k = 2.*sc.pi/λ_0
ω = omega(k, ϕ + .5*sc.pi, n_h, dt_range, dx_strict, dy_strict, dz_strict)

In [ ]:
figure(figsize=(8,6))

plot(ω/k/sc.c - 1., "o-")

xlabel(r"Number of time step")
ylabel(r"$\tilde v_p/c - 1$")
title("Velocity error depending on timestep\nλ=%inm, n_h=%i, ϕ=%.1f°"%(2.*sc.pi/k*1.e9, n_h, ϕ*180./sc.pi))

#legend()
grid()

#xticks(np.arange(0.1,1+0.1,0.1))
#xlim(0,90.)
#ylim(1.e-4, 2.e-2)
yscale('log')

show()
close()

In [ ]:
# Calculate Laser propagation length over which
# the impact of numerical dispersion must be negligible
L_trans_arr = array([.5 * L_z, z_r])
i_trans = L_trans_arr.argmin()
L_trans = L_trans_arr[i_trans]
print("Transition length: index = %i, L_trans = %e"%(i_trans, L_trans))
L_cyc = L_trans / sin(ϕ)
print("Laser propagation distance over which dispersion is of interest = %.3fmm"%(L_cyc*1.e3))
t_cyc = L_cyc / sc.c # corresponding time frame
sim_steps = t_cyc/dt_range
print("simulation steps depending on step size =", sim_steps)

figure(figsize=(8,6))

prop_deviation = abs(ω/k/sc.c - 1.) * sc.c * sim_steps * dt_range / λ_plasma
plot(prop_deviation, "o-")

xlabel(r"Number of time step")
ylabel(r"$\Delta v * t_{tot} / \lambda_{plasma}$")
title("Propagation distance deviation after %.2eΔt_max\nλ=%.2fµm, n_h=%i, ϕ=%.1f°"%(t_cyc/dt_max, 2.*sc.pi/k*1.e6, n_h, ϕ*180./sc.pi))

#legend()
grid()

#xticks(np.arange(0.1,1+0.1,0.1))
#xlim(0,90.)
#ylim(0., 1.)
#yscale('log')

show()
close()

In [ ]:
# Choose time step
chosen_index = 8
dt_strict = dt_range[chosen_index]
print("dt_strict = %.6e"%(dt_strict))
print("propagation distance deviation after %.6eΔt = %.2e"%(sim_steps[chosen_index], prop_deviation[chosen_index]))

target_energy = 100.e9 # unit: eV
E_acc = 1.e11 # accelerating field, unit: eV/m
z_acc = target_energy / E_acc # acceleration distance, unit: m
t_tot = z_acc / sc.c # acceleration time, unit: s
simulation_time = t_tot / dt_strict
print("Total simulation time = %iMΔt"%(simulation_time*1.e-6))

#### Sampling summary

In [ ]:
print("sanity check: temporal samples per laser period λ_Laser / c Δt =", λ_0 / sc.c / dt_strict)
print("sanity check: Δy / c Δt =", dy_strict / sc.c / dt_strict)
print("sanity check CFL condition: c Δt =", sc.c * dt_strict, "<", 1. / aofdtd /sqrt( 1./dx_strict**2 + 1./dy_strict**2 + 1./dz_strict**2 ), "!?")

In [ ]:
print("Summary of sampling parameters")
print("==============================")
print("dt = %.5es = %.3f T_Laser = %.1e T_Plasma"%(dt_strict, dt_strict/(λ_0 / sc.c), dt_strict/(2. * sc.pi / PLASMA_FREQUENCY_SI)))
print("dx = %.5em = %.3f λ_Laser = %.1e λ_Plasma"%(dx_strict, dx_strict/λ_0, dx_strict/λ_plasma_base))
print("dy = %.5em = %.3f λ_Laser = %.1e λ_Plasma"%(dy_strict, dy_strict/λ_0, dy_strict/λ_plasma_base))
print("dz = %.5em = %.3f λ_Laser = %.1e λ_Plasma"%(dz_strict, dz_strict/λ_0, dz_strict/λ_plasma_base))

In [ ]:
print("Typical number of real electrons per cell =", BASE_DENSITY_SI * dx_strict * dy_strict * dz_strict)

### Evaluate correct initial temperature

In [ ]:
used_density = BASE_DENSITY_SI
T_min = dx_strict**2 * used_density * sc.e / sc.epsilon_0 # in [eV]
print("Minimum intialization temperature to circumvent finite grid instability at initial preionization T_min = %.4ekeV"%(T_min * 1.e-3))

T_init = 0. #T_min
print("Initialization Temperature T_init = %.4ekeV"%(T_init*1.e-3))

λ_D = sqrt(sc.epsilon_0 * T_init / ( used_density * sc.e ) )
print("Debye length @ T_init λ_D(T_init) / Δx = %.4e"%(λ_D / dx_strict))

### Simulation window volume

In [ ]:
box_cells_x_raw = int(L_x / dx_strict)
print("box L_x: ",box_cells_x_raw)
box_cells_y_raw = int(L_y / dy_strict)
print("box L_y: ",box_cells_y_raw)
box_cells_z_raw = int(L_z / dz_strict)
print("box L_z: ",box_cells_z_raw)

cells_norm = box_cells_x_raw
print("Ratio of box dimensions [cells] : x : y : z = %.1f : %.1f : %.1f"%(box_cells_x_raw/cells_norm, box_cells_y_raw/cells_norm, box_cells_z_raw/cells_norm))

### Calculate the real number of cells in the box

Aim for a simulation volume per GPU that has an approximately even number of cells in all directions.

Berechne Verhältnis der Boxlängen zueinander und nimm selbes Verhältnis für Verteilung der Grafikkarten.

#### Estimate GPU distribution from ratio of box dimensions

In [ ]:
# domain decomposition
gpus_x = 1
gpus_y = 8
gpus_z = 2

print("GPUs x : y : z = %.1f : %.1f : %.1f"%(gpus_x, gpus_y, gpus_z))
print("total number of gpus:", gpus_x * gpus_y *gpus_z)

# super cell
super_cell = np.array([8, 8, 8])

box_raw = np.array([box_cells_x_raw, box_cells_y_raw, box_cells_z_raw])
gpu_raw = np.array([box_raw[0] / gpus_x / super_cell[0]
                    , box_raw[1] / gpus_y / super_cell[1]
                    , box_raw[2] / gpus_z / super_cell[2]])

print("simulation box size raw =", box_raw)
print("super cells per gpu:", gpu_raw)

In [ ]:
box_cells_x = 28 * gpus_x * super_cell[0]
box_cells_y = 40 * gpus_y * super_cell[1]
box_cells_z = 28 * gpus_z * super_cell[2]

box = np.array([box_cells_x, box_cells_y, box_cells_z], dtype=int)
print("Simulation box size = ", box)
window = box / array([gpus_x, gpus_y, gpus_z]) * array([gpus_x, gpus_y-1, gpus_z])
print("Moving window size = ", window)
print("box dimesion per gpu (x, y, z) [cells] = (%.1f, %.1f, %.1f)"%(box_cells_x / gpus_x, box_cells_y / gpus_y, box_cells_z / gpus_z))

In [ ]:
print("Laser start position focus_y = %iDy_SI"%(int(window[1]*0.75)))

## Estimate memory consumption

In [ ]:
# typical number of particles per cell
N_PPC = 25.

print("Number of particles = %.5e"%(box[0]*box[1]*box[2]*N_PPC))

In [ ]:
n_x = int(box_cells_x / gpus_x)
n_y = int(box_cells_y / gpus_y)
n_z = int(box_cells_z / gpus_z)

mc = MemoryCalculator(
    n_x=n_x,
    n_y=n_y,
    n_z=n_z,
    precision_bits=32
)

In [ ]:
print("Memory requirement per GPU:")
# field memory per GPU
# * 2 Additional field in fileOutput.param: chargeDensity and energyDensity
# * 6 additional fields due to usage of brackgroundfield implementation with double buffering
field_gpu = mc.mem_req_by_fields(field_tmp_slots=2+6,particle_shape_order=3,sim_dim=3)
print("+ fields: {:.2f} MB".format(
    field_gpu / (1024 * 1024)))
# particle memory per GPU - only the target area contributes here
e_gpu = mc.mem_req_by_particles(
    n_x, n_y, n_z, num_additional_attributes = 0,
    particles_per_cell = N_PPC)

# memory for calorimeters
cal_gpu = mc.mem_req_by_calorimeter(n_energy=1024, n_yaw=180, n_pitch=90
                                     )

print("+ species:")
print("- e: {:.2f} MB".format(e_gpu / (1024 * 1024)))

rng_gpu = mc.mem_req_by_rng(n_x, n_y, n_z)
print("+ RNG states: {:.2f} MB".format(rng_gpu / (1024 * 1024)))

print("+ particle calorimeters: {:.2f} MB".format(cal_gpu / (1024 * 1024)))

mem_sum = field_gpu + e_gpu + rng_gpu + cal_gpu
print("Sum of required GPU memory: {:.2f} MB".format(mem_sum / (1024 * 1024)))

**Available clusters**

* taurus: *gpu2* partition: 256 x Tesla **k80 with 11441MiB memory**
* taurus: *ml* partition: 132 x Tesla **V100 with 32256MiB memory**
* Summit: partition: 27648 Nvidia Tesla **V100 with 16130MiB memory** (4608 nodes, in total 9216 IBM POWER9 CPUs and 27648 Nvidia Tesla GPUs)
* hemera: *fwkt_v100* partition: 16 x 4 Tesla **V100 with 32256MiB memory**
* spock: *caar* partition: 4 x 4 AMD **MI100 with 32GiB memory** (for 3 hours), 4 x 16 GPU/s for one hour

# Creating the example 

Branch: `setup-2021-02_TWEAC-PRX-for-CAAR`

## `grid.param`

In [ ]:
print("dt = {:.5e} s".format(dt_strict))
print("dx = {:.5e} m".format(dx_strict))
print("dy = {:.5e} m".format(dy_strict))
print("dz = {:.5e} m".format(dz_strict))

## `fieldSolver.param`

In [ ]:
print("using Solver = maxwellSolver::ArbitraryOrderFDTDPML<%i>;"%(n_h))

## `density.param`

## Gas profiles

In [ ]:
def richard_density(y, z):
    CELL_WIDTH_SI = dx_strict
    CELL_HEIGHT_SI = dy_strict
    CELL_DEPTH_SI = dz_strict
    
    VACUUM_Y_SI = 100.0 * CELL_HEIGHT_SI
    VACUUM_Z_SI = 148.0 * CELL_DEPTH_SI
    Z_GRADIENT_SI = 32.0 * CELL_DEPTH_SI

    SIM_BOX_Z = box[2] * CELL_DEPTH_SI # MATCH THIS WITH Z-COMPONENT OF "TBG_gridsize" IN CFG_FILE !!!                                                              
    
    PLASMA_Z_LENGTH_SI = SIM_BOX_Z - 2.0 * VACUUM_Z_SI
    
    INITIAL_UPRAMP = 0.0e-6 ##change
    LOW_DENSITY_PLATEAU = 0.0e-6 ##change
    TORCH_UPRAMP = 120.0e-6 ##change
    HIGH_DENSITY_PLATEAU = 240.0e-6 ##change
    DOWNRAMP_1 = 15.0e-6 ##change
    DOWNRAMP_2 = 0.0e-6 ##change
    
    REL_DENSITY_LOW = 0.0
    REL_DENSITY_HIGH = 2.0
    REL_DENSITY_MEDIUM = 1.0 #sqrt(2.0)
    REL_DENSITY_BASE = 1.0 # always one, use BASE_DENSITY_SI to change base density
                           # just here to avoid "magic" 1.0s in the code
    
    REL_DENSITY = REL_DENSITY_BASE
    
    
    # Plot of density profile                                                                  
    #
    #   density                                                                                  
    #
    # REL_DENSITY_HIGH   |        ___                                                                              
    # REL_DENSITY_MEDIUM |       /   \                                                                             
    # REL_DENSITY_LOW    |    __/     `.                                                                           
    # REL_DENSITY_BASE   |   /          `._______                                                                  
    #                    |  /                                                                                    
    #                    |----------------------------  y                                                       
    #                      A B C D E  F  G   H                                                                   
    # */
    
    
    # vacuum before gas (A)                                                                    
    if ( y < VACUUM_Y_SI ):
        return 0.0
    if ( z < VACUUM_Z_SI or z >= (PLASMA_Z_LENGTH_SI + VACUUM_Z_SI ) ):
        REL_DENSITY = 0.0
    if ( z >= VACUUM_Z_SI and z < ( VACUUM_Z_SI + Z_GRADIENT_SI ) ):
        REL_DENSITY =  (z - VACUUM_Z_SI ) / Z_GRADIENT_SI
    if ( z < ( PLASMA_Z_LENGTH_SI + VACUUM_Z_SI ) and z >= ( PLASMA_Z_LENGTH_SI + VACUUM_Z_SI - Z_GRADIENT_SI ) ):
        REL_DENSITY = ( PLASMA_Z_LENGTH_SI + VACUUM_Z_SI - z) / Z_GRADIENT_SI
    
    # first up-ramp (B)
    y_B = VACUUM_Y_SI + INITIAL_UPRAMP
    if ( y >= ( VACUUM_Y_SI ) and y < y_B ):
        return REL_DENSITY * REL_DENSITY_LOW * ( ( y - VACUUM_Y_SI ) / INITIAL_UPRAMP )
    
    # first plateau (C)
    y_C = y_B + LOW_DENSITY_PLATEAU
    if ( y >= y_B and y < y_C ):
        return REL_DENSITY * REL_DENSITY_LOW
    
    # second up-ramp (D)
    y_D = y_C + TORCH_UPRAMP
    if ( y >= y_C and y < y_D ):
        return REL_DENSITY * (REL_DENSITY_HIGH - REL_DENSITY_LOW) * ( ( y - y_C ) / TORCH_UPRAMP ) + REL_DENSITY_LOW
    
    # second plateau (E)
    y_E = y_D + HIGH_DENSITY_PLATEAU
    if ( y >= y_D and y < y_E ):
        return REL_DENSITY * REL_DENSITY_HIGH
    
    # first down-ramp (F)
    y_F = y_E + DOWNRAMP_1
    if ( y >= y_E and y < y_F ):
        return REL_DENSITY * (REL_DENSITY_MEDIUM - REL_DENSITY_HIGH) * ( y - y_E ) / DOWNRAMP_1 + REL_DENSITY * REL_DENSITY_HIGH

    # second down-ramp (G)
    y_G = y_F + DOWNRAMP_2
    if ( y >= y_F and y < y_G ):
        return REL_DENSITY * ( REL_DENSITY_BASE - REL_DENSITY_MEDIUM ) * ( y - y_F ) / DOWNRAMP_2 + REL_DENSITY * REL_DENSITY_MEDIUM
        
    # third plateau (H)
    if ( y >= y_G):
        return REL_DENSITY * REL_DENSITY_BASE # 0.32+e19 cm^-3                                         

    return (REL_DENSITY * REL_DENSITY_BASE)

In [ ]:
def density_from_array(y,z):
    """ since the density profile functions above only work fpr single (y,z)-pairs
        this wrapper function allows to calculate the density in a (y,z)-plane from
        arrays of y- and z-values.
        
        NOTE: Adjust the density you want to plot by using the respective function in
        the line starting with `density[j,i] = ...`
    """
    density = zeros((len(z),len(y)))
    #print(shape(density))
    j=0
    for z_j in z:
        i=0
        for y_i in y:
            density[j,i] = richard_density(y_i, z_j) ## linear increase: 10.*y_i #2.*z_j 
                                                     ## Alex original: alex_density(y_i, z_j)
            #print(j,i)
            #print(y_i,z_j)
            i+=1
        j+=1
    
    return density

In [ ]:
y = np.linspace(0., 400.e-6, 200)
z = np.linspace(0., box[2]*dz_strict, 200)
density2d = density_from_array(y,z)

In [ ]:
figure(figsize=((8,6)))

imshow(density2d
       #, extent=(y[0]*1.e6, y[-1]*1.e6, z[0]*1.e6, z[-1]*1.e6)
       , extent=(y[0]/dy_strict, y[-1]/dy_strict, z[0]/dz_strict, z[-1]/dz_strict)
       , interpolation='none'
       , origin='lower'
       #, vmin = , vmax = 
       , cmap=cm.plasma
       , aspect='auto'
)

xlabel(r"y")
ylabel(r"z")

colorbar()

show()

In [ ]:
figure((figsize(16,6)))
subplot(1,2,1)

plot(y*1.e6, density2d[len(z)//2,:])

xlabel("y [mum]")
#xlim(0,500)
#xticks(arange(0,501,25))

ylabel("rel. Density")
grid()


subplot(1,2,2)
plot(y/dy_strict, density2d[len(z)//2,:])
xlabel("y [cells]")
grid()

show()

## `species.param`

```C++
    using UsedParticleShape = particles::shapes::PQS;

    using UsedParticleCurrentSolver = currentSolver::EmZ< UsedParticleShape >;

    using UsedParticlePusher = particles::pusher::HigueraCary;
```

## `speciesDefinition.param`

Deleted everything except electrons.

## `particle.param`

Adjust minimum weighting and numbers of particles per cell.

```C++
    /** a particle with a weighting below MIN_WEIGHTING will not
     *      be created / will be deleted
     *
     *  unit: none */
    constexpr float_X MIN_WEIGHTING = 10;

    /** Number of maximum particles per cell during density profile evaluation.
     *
     * Determines the weighting of a macro particle and with it, the number of
     * particles "sampling" dynamics in phase space.
     */
    constexpr uint32_t TYPICAL_PARTICLES_PER_CELL = 25u;

```

In [ ]:
print("Typical number of real electrons per cell =", BASE_DENSITY_SI * dx_strict * dy_strict * dz_strict)

In [ ]:
print("current TYPICAL_PARTICLES_PER_CELL =", N_PPC)

## `particleFilters.param`

Added something according to old run on Piz Daint.

## `speciesInitialization.param`

Set density profile, start position, add drift and temperature.

```C++
using InitPipeline = bmpl::vector<
        CreateDensity<
            densityProfiles::FreeFormula,
            startPosition::Random,
            PIC_Electrons
        >,
        Manipulate<
            manipulators::AssignYDriftPositive,
            PIC_Electrons
        >,
        Manipulate<
            manipulators::AddTemperature,
            PIC_Electrons
        >
    >;
```

## `memory.param`

Increase default exchange memory and introduce electron exchange memory
```C++
/* short namespace*/
    namespace mCT = pmacc::math::CT;
    /** size of a superCell
     *
     * volume of a superCell must be <= 1024
     */
    using SuperCellSize = typename mCT::shrinkTo<mCT::Int<8, 8, 8>, simDim>::type;


struct DefaultExchangeMemCfg
    {
        // memory used for a direction
        static constexpr uint32_t BYTES_EXCHANGE_X = 1 * 1024 * 1024; // 1 MiB
        static constexpr uint32_t BYTES_EXCHANGE_Y = 3 * 1024 * 1024; // 3 MiB
        static constexpr uint32_t BYTES_EXCHANGE_Z = 1 * 1024 * 1024; // 1 MiB
        static constexpr uint32_t BYTES_EDGES = 32 * 1024; // 32 kiB
        static constexpr uint32_t BYTES_CORNER = 8 * 1024; // 8 kiB

        /** Reference local domain size
         *
         * The size of the local domain for which the exchange sizes `BYTES_*` are configured for.
         * The required size of each exchange will be calculated at runtime based on the local domain size and the
         * reference size. The exchange size will be scaled only up and not down. Zero means that there is no reference
         * domain size, exchanges will not be scaled.
         */
        using REF_LOCAL_DOM_SIZE = mCT::Int<0, 0, 0>;
        /** Scaling rate per direction.
         *
         * 1.0 means it scales linear with the ratio between the local domain size at runtime and the reference local
         * domain size.
         */
        const std::array<float_X, 3> DIR_SCALING_FACTOR = {{0.0, 0.0, 0.0}};
    };

    struct ElectronExchangeMemCfg
    {
        // memory used for a direction
        static constexpr uint32_t BYTES_EXCHANGE_X = 4 * 1024 * 1024; // 4 MiB
        static constexpr uint32_t BYTES_EXCHANGE_Y = 96 * 1024 * 1024; // 96 MiB
        static constexpr uint32_t BYTES_EXCHANGE_Z = 4 * 1024 * 1024; // 4 MiB
        static constexpr uint32_t BYTES_EDGES = 256 * 1024; // 256 kiB
        static constexpr uint32_t BYTES_CORNER = 256 * 1024; // 256 kiB

        /** Reference local domain size
         *
         * The size of the local domain for which the exchange sizes `BYTES_*` are configured for.
         * The required size of each exchange will be calculated at runtime based on the local domain size and the
         * reference size. The exchange size will be scaled only up and not down. Zero means that there is no reference
         * domain size, exchanges will not be scaled.
         */
        using REF_LOCAL_DOM_SIZE = mCT::Int<0, 0, 0>;
        /** Scaling rate per direction.
         *
         * 1.0 means it scales linear with the ratio between the local domain size at runtime and the reference local
         * domain size.
         */
        const std::array<float_X, 3> DIR_SCALING_FACTOR = {{0.0, 0.0, 0.0}};
    };
```

## `fieldbackground.param`
Add 4 TWTS pulses per field (E, B) in order to generate 2 pulse-front titled laser pulses with crossed polarization.
The polarization direction calculations for the individual pulses was done by @beyondEspresso.

### E-Field
Abstracted parameters of E- and B-Field into single struct in `TwtsBackgroundLaser.param`.

### B-Field
Exactly the same as E-Field

## `TwtsBackgroundLaser.param`

In [ ]:
print("static constexpr float_64 WAVE_LENGTH_SI = %.3e;"%(λ_0))
print("static constexpr float_64 _A0  = %.2f;"%(_A0))
print("static constexpr float_64 PULSE_LENGTH_SI = %e / 2.354820045;"%(τ_FWHM_I))
print("static constexpr float_64 W0_SI = %.2e;"%(w_0x))
FOCUS_Y_Dy = int(window[1]*0.75)
print("static constexpr float_64 FOCUS_Y_SI = %i * SI::CELL_HEIGHT_SI;"%(FOCUS_Y_Dy))
print("static constexpr float_64 PHI = %.2f * (PI/180.);"%(ϕ*180./sc.pi))
TDELAY = 0.
print("static constexpr float_64 TDELAY = %e;"%(TDELAY))
vy = 1. / sqrt(1. + cos(2.*ϕ))
vx = sqrt(1. - vy**2)
print("static constexpr float_X NORM_E_B_12 = %f;"%(vx))
print("static constexpr float_X NORM_E_B_34 = %f;"%(vy))

## `png.param`

Change channel settings
```
    /* png preview settings for each channel */
    DINLINE float_X preChannel1(const float3_X& field_B, const float3_X& field_E, const float3_X& field_J)
    {
        return math::abs2(field_J);
    }

    DINLINE float_X preChannel2(const float3_X& field_B, const float3_X& field_E, const float3_X& field_J)
    {
        return field_E.x() * field_E.x() + field_E.y() * field_E.y() + field_E.z() * field_E.z();
    }
 
    DINLINE float_X preChannel3(const float3_X& field_B, const float3_X& field_E, const float3_X& field_J)
    {
       return -1.0_X * field_E.y();
    }

```

<span style="font-size:xx-large; font-weight:bold; color:coral">What about?</span>

```C++
    constexpr float_64 scale_image = 1./64.;
```

# Config files

### FOM run

Only:
* periodic boundary condition
* macroparticle counter

In [ ]:
print("Simulation box size = ", box)

In [ ]:
print(2.*sc.pi/PLASMA_FREQUENCY_SI/dt_strict)

In [ ]:
print(14./15.)

In [ ]:
phi = 7.*sc.pi/ 180.
vy = 1 / sqrt(1. + cos(2.*phi))
vx = sqrt(1. - vy**2)
print(vy/0.7084281424758874, vx/0.7057829460593135)